# Collecting Personal Fitness Data from the Fitbit API 

This is a step-by-step guide to collecting personal fitness data from the Fitbit API. <br>
<br>
For this, you first need to:
- have a Fitbit account
- have personal data (or access to other user data)
- have a Fitbit app and access to its client ID and secret token

(I might add instructions on how to create a personal Fitbit app and look up client ID and secret later. For now, you can follow the instructions on this [tutorial](https://towardsdatascience.com/collect-your-own-fitbit-data-with-python-ff145fa10873)).

## 1. Imports and Authentication

The data collection described here utilises the python interface for handling the Fitbit API.<br>
- This is available for free on GitHub and can be accessed through this [link](https://github.com/orcasgit/python-fitbit). <br>
- Once you have downloaded the repo and unzipped the files, 
- Open the command line, change the directory to the directory containing the unzipped repo files and run: 'sudo pip install -r requirements/base.txt'. This ensures all of the requirements to running the python Fitbit API are met before proceeding.

__*In the api.py, I have hard coded the API_VERSION to 1.2 (lines 23 and 190). This gives access to more details, including sleep stages for sleep data.*__

In [4]:
import fitbit # API_VERSION was hard coded to 1.2
import gather_keys_oauth2 as Oauth2

import pandas as pd
import numpy as np
import datetime

import csv

In [ ]:
CLIENT_ID = 'your_CLIENT_ID'
CLIENT_SECRET = 'your_CLIENT_SECRET'

In [ ]:
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])

auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, 
                             access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

## 2. Sleep Data Collection

### 2.1 Collection of sleep stage and night time details

In [10]:
# collect sleep stage data from past x days and save in csv files
# insert date of last collected record
last_record = datetime.date(2018, 4, 14)

# today's date
today_date = datetime.date.today()

# number of days between last record and today
delta = today_date - last_record
delta_days = delta.days


for i in range(0, delta_days):
    date = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y-%m-%d'))
    date_file = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y%m%d'))
    
    sleep = auth2_client.sleep(date=date)['sleep']
    
    for s in range(0, len(sleep)):
        try:
            sltimestages = auth2_client.sleep(date=date)['sleep'][s]['levels']['data']
            
            sltime_list = []
            slstage_list = []
        
            for i in sltimestages:
                sltime_list.append(i['dateTime'][-12:-4])
                slstage_list.append(i['level'])

            sleep_df = pd.DataFrame({'Stage':slstage_list,
                                     'Time':sltime_list})

            sleep_df.to_csv('./downloads/sleep_detailedstages/sleepstages' + \
                            date_file + '_' + str(s) + '.csv',
                            header=True,
                            index=False)
        
            print('File sleepstages{}_{}.csv successfully created.'.format(date_file, s))
    
        except IndexError:
            print('No sleep records for {}'.format(date))
            continue

File sleepstages20180505_0.csv successfully created.
File sleepstages20180504_0.csv successfully created.
File sleepstages20180503_0.csv successfully created.
File sleepstages20180503_1.csv successfully created.
File sleepstages20180502_0.csv successfully created.
File sleepstages20180502_1.csv successfully created.
File sleepstages20180501_0.csv successfully created.
File sleepstages20180430_0.csv successfully created.
File sleepstages20180429_0.csv successfully created.
File sleepstages20180428_0.csv successfully created.
File sleepstages20180427_0.csv successfully created.
File sleepstages20180426_0.csv successfully created.
File sleepstages20180425_0.csv successfully created.
File sleepstages20180424_0.csv successfully created.
File sleepstages20180423_0.csv successfully created.
File sleepstages20180421_0.csv successfully created.
File sleepstages20180420_0.csv successfully created.
File sleepstages20180419_0.csv successfully created.
File sleepstages20180418_0.csv successfully cr

### 2.2 Collection of sleep summary stats
#### 2.2.1 Initial run to create dataframe and export to csv
- Data from each sleep log for every date in time period are saved into a new row.
- Need to run only once to get csv file. Then use subsequent method for appending rows for new sleep logs.

In [44]:
# collect sleep stats for a period of time
beginning_time = datetime.date(2018, 4, 14)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - beginning_time
delta_days = delta.days


for i in range(0, delta_days):
    date = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y-%m-%d'))
   
    sleep = auth2_client.sleep(date=date)['sleep']
    
    # sleep records may not be present for every day, so if there's an IndexError, we'll skip that date
    for s in range(0, len(sleep)):
        try:
            sleep_stats = sleep[s]
            sleep_stages = sleep[s]['levels']['summary']
        
        except IndexError:
            print('No sleep records for {}'.format(date))
            pass
    
        # take stats into dataframe
        # if record is simplified (no stages), we'll skip it with except keyerror below
        try:
            slsummary_df = pd.DataFrame({
                'Date':sleep_stats['dateOfSleep'],
                'MainSleep':sleep_stats['isMainSleep'],
                'Efficiency':sleep_stats['efficiency'],
                'Duration':sleep_stats['duration'],
                'Minutes Asleep':sleep_stats['minutesAsleep'],
                'Minutes Awake':sleep_stats['minutesAwake'],
                'Time in Bed':sleep_stats['timeInBed'],
                'Sleep_Deep':sleep_stages['deep']['minutes'],
                'Sleep_Light':sleep_stages['light']['minutes'],
                'Sleep_REM':sleep_stages['rem']['minutes'],
                'Sleep_stage_Wake':sleep_stages['wake']['minutes'],
                'Sleep_stage_Wake_Count':sleep_stages['wake']['count']
            } ,index=[0])
    
            # combine all stats in single dataframe
            if i == 0:
                slsummaryall_df = slsummary_df
            else:
                slsummaryall_df = pd.concat([slsummary_df, slsummaryall_df])
            
        except KeyError:
            print('Incomplete sleep records for {}, index {}'.format(date, s))
            pass


Incomplete sleep records for 2018-05-03, index 1
Incomplete sleep records for 2018-05-01, index 0
Incomplete sleep records for 2018-04-30, index 0
Incomplete sleep records for 2018-04-19, index 0
Incomplete sleep records for 2018-04-17, index 0


In [45]:
# add day of the week to the dataframe
slsummaryall_df['Date'] = pd.to_datetime(slsummaryall_df['Date'])
slsummaryall_df['DOW'] = slsummaryall_df['Date'].dt.weekday_name

In [47]:
cols_ordered_slsdetails = ['Date',
                           'DOW',
                           'Duration', # milliseconds
                           'Efficiency',
                           'MainSleep',
                           'Minutes Asleep',
                           'Minutes Awake',
                           'Sleep_Deep',
                           'Sleep_Light',
                           'Sleep_REM',
                           'Sleep_stage_Wake',
                           'Sleep_stage_Wake_Count', 
                           'Time in Bed', 
                          ]
slsummaryallordered_df = slsummaryall_df[cols_ordered_slsdetails]

In [48]:
slsummaryallordered_df

,Date,DOW,Duration,Efficiency,MainSleep,Minutes Asleep,Minutes Awake,Sleep_Deep,Sleep_Light,Sleep_REM,Sleep_stage_Wake,Sleep_stage_Wake_Count,Time in Bed
0,2018-04-15,Sunday,26820000,97,True,398,49,57,277,64,49,13,447
0,2018-04-16,Monday,33000000,97,True,464,86,71,319,74,86,23,550
0,2018-04-18,Wednesday,35040000,92,True,508,76,36,399,73,76,25,584
0,2018-04-20,Friday,30420000,95,True,433,74,31,342,60,74,33,507
0,2018-04-21,Saturday,29520000,94,True,411,81,48,309,54,81,22,492
0,2018-04-23,Monday,28500000,94,True,401,74,87,206,108,74,18,475
0,2018-04-24,Tuesday,41280000,95,True,562,126,48,453,61,126,32,688
0,2018-04-25,Wednesday,36840000,94,True,507,107,77,403,27,107,30,614
0,2018-04-26,Thursday,34260000,94,True,506,65,42,414,50,65,22,571
0,2018-04-27,Friday,38280000,92,True,556,82,35,441,80,82,37,638


In [49]:
# save sleep stats into csv
slsummaryallordered_df.to_csv('./downloads/slsummarystagesall.csv',
                              header=True, index=False)

#### 2.2.2 Periodic run to append new sleep logs to exported csv
- Data from each sleep log for every date in time period are saved into a new row.
- Use this for new sleep logs.

In [11]:
# collect sleep stats and append to existing csv

# insert date for last collected record
last_record = datetime.date(2018, 5, 3)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - last_record
delta_days = delta.days

# range does not include date of last record and iterates from oldest date to newest
for i in range(delta_days-1, -1, -1):
    date_dt = datetime.datetime.now() - datetime.timedelta(days = i)
    date = str(date_dt.strftime('%Y-%m-%d'))
    
    sleep = auth2_client.sleep(date=date)['sleep']
    
    # sleep records may not be present for every day, so if there's an IndexError, we'll skip that date
    for s in range(0, len(sleep)):
        try:
            sleep_stats = sleep[s]
            sleep_stages = sleep[s]['levels']['summary']
        
        except IndexError:
            print('No sleep records for {}'.format(date))
            pass
    
        # take stats into list
        # if record is simplified (no stages), we'll skip it with except keyerror below
        try:
            slstats_list = []
        
            slstats_list.append(sleep_stats['dateOfSleep'])
            slstats_list.append(date_dt.strftime('%A'))
            slstats_list.append(sleep_stats['duration'])
            slstats_list.append(sleep_stats['efficiency'])
            slstats_list.append(sleep_stats['isMainSleep'])
            slstats_list.append(sleep_stats['minutesAsleep'])
            slstats_list.append(sleep_stats['minutesAwake'])
            slstats_list.append(sleep_stages['deep']['minutes'])
            slstats_list.append(sleep_stages['light']['minutes'])
            slstats_list.append(sleep_stages['rem']['minutes'])
            slstats_list.append(sleep_stages['wake']['minutes'])
            slstats_list.append(sleep_stages['wake']['count'])
            slstats_list.append(sleep_stats['timeInBed'])
        
            # convert list to numpy array and transpose
            slstats_np = np.asarray(slstats_list)
            new_csv_row = slstats_np.T
        
            # add new row to existing csv
            with open(r'./downloads/slsummarystagesall.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(new_csv_row)
                print('Row added for sleep from {}, index {}'.format(date, s))
    
        except KeyError:
            print('Incomplete sleep records for {}, index {}'.format(date, s))
            pass
    

Row added for sleep from 2018-05-04, index 0
Row added for sleep from 2018-05-05, index 0


## 3. Heart Rate Data Collection
### 3.1 Intraday Heart Rate Data Collection
- Heart rate data (1 sec detail level) for every date in time period are saved into a new csv file.

In [12]:
# collect intraday heart rate data from past x days and save in csv files
# insert date of last collected record
last_record = datetime.date(2018, 5, 2)

# today's date
today_date = datetime.date.today()

# number of days between last record and today
delta = today_date - last_record
delta_days = delta.days

# note range here is from 1 to delta_days as current day isn't complete
for i in range(1, delta_days):
    date = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y-%m-%d'))
    date_file = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y%m%d'))
    act_heart = auth2_client.intraday_time_series('activities/heart',
                                                  base_date=date,
                                                  detail_level='1sec')
    time_list = []
    val_list = []

    for i in act_heart['activities-heart-intraday']['dataset']:
        val_list.append(i['value'])
        time_list.append(i['time'])
    
    heart_df = pd.DataFrame({'Heart Rate':val_list, 'Time':time_list})
    heart_df.to_csv('./downloads/heart/heart'+\
               date_file + '.csv', \
               columns=['Time', 'Heart Rate'], header=True, \
               index = False)

## 4. Activity Data Collection
### 4.1 Daily Stats Data Collection
#### 4.1.1 Initial run to create dataframe and export to csv
- Data for each day in time period are saved into a new row.
- Need to run only once to get csv file. Then use subsequent method for appending rows for new daily logs.

In [53]:
# collect activity summary stats for a period of time
beginning_time = datetime.date(2018, 4, 14)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - beginning_time
delta_days = delta.days


for i in range(1, delta_days):
    date = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y-%m-%d'))
    
    # activity records may not be present for every day, so if there's an IndexError, we'll skip that date
    try:
        act_stats = auth2_client.activities(date=date)['summary']
        HR_stats = auth2_client.intraday_time_series('activities/heart', base_date=date)
    except IndexError:
        print('No activity summary records for {}'.format(date))
        pass
    
    # take stats into dataframe
    try:
        acsummary_df = pd.DataFrame({
            'Date':date,
            
            'Steps':act_stats['steps'],
            'Floors':act_stats['floors'],
            'Elevation_foot':act_stats['elevation'], 
            'Distance_mile':act_stats['distance'],

            'Calories_total':act_stats['calories']['total'],
            'Calories_bmr':act_stats['calories']['bmr'],
            
            'Sedentary_min':act_stats['activityLevels'][0]['minutes'],
            'Light_min':act_stats['activityLevels'][1]['minutes'],
            'Moderate_min':act_stats['activityLevels'][2]['minutes'],
            'Very_min':act_stats['activityLevels'][3]['minutes'],
            
            'HR_outrange_min':act_stats['heartRateZones'][0]['minutes'],
            'HR_fatburn_min':act_stats['heartRateZones'][1]['minutes'],
            'HR_cardio_min':act_stats['heartRateZones'][2]['minutes'],
            'HR_peak_min':act_stats['heartRateZones'][3]['minutes'],
            
            'HR_outrange_cal':act_stats['heartRateZones'][0]['caloriesOut'],
            'HR_fatburn_cal':act_stats['heartRateZones'][1]['caloriesOut'],
            'HR_cardio_cal':act_stats['heartRateZones'][2]['caloriesOut'],
            'HR_peak_cal':act_stats['heartRateZones'][3]['caloriesOut'],
            
            'HR_meanresting':HR_stats['activities-heart'][0]['value']['restingHeartRate']
            
        } ,index=[0])
    
        # combine all stats in single dataframe
        if i == 1:
            acsummaryall_df = acsummary_df
        else:
            acsummaryall_df = pd.concat([acsummary_df, acsummaryall_df])
            
    except KeyError:
        print('Incomplete activity records for {}'.format(date))
        pass


In [54]:
# add day of the week to the dataframe
acsummaryall_df['Date'] = pd.to_datetime(acsummaryall_df['Date'])
acsummaryall_df['DOW'] = acsummaryall_df['Date'].dt.weekday_name

In [55]:
cols_ordered = ['Date',
                'DOW',
                'Calories_total',
                'Calories_bmr',
                'Steps',
                'Floors',
                'Distance_mile',
                'Elevation_foot',
                'Sedentary_min',
                'Light_min',
                'Moderate_min',
                'Very_min',
                'HR_meanresting',
                'HR_outrange_cal', # HR out of range: min 30, max 96
                'HR_outrange_min',
                'HR_fatburn_cal', # HR fat burn: min 96, max 135
                'HR_fatburn_min',
                'HR_cardio_cal', # HR cardio: min 135, max 164
                'HR_cardio_min',
                'HR_peak_cal', # HR peak: min 164, max 220
                'HR_peak_min'
               ]
acsummaryallordered_df = acsummaryall_df[cols_ordered]

In [56]:
acsummaryallordered_df

,Date,DOW,Calories_total,Calories_bmr,Steps,Floors,Distance_mile,Elevation_foot,Sedentary_min,Light_min,...,Very_min,HR_meanresting,HR_outrange_cal,HR_outrange_min,HR_fatburn_cal,HR_fatburn_min,HR_cardio_cal,HR_cardio_min,HR_peak_cal,HR_peak_min
0,2018-04-15,Sunday,2451,1359,5174,10,1.86,100,466,288,...,30,53,1493.31360,1227,901.14240,205,56.64000,8,0,0
0,2018-04-16,Monday,1887,1359,3487,25,1.14,250,536,160,...,11,52,1604.13920,1377,269.60640,61,13.78240,2,0,0
0,2018-04-17,Tuesday,2113,1355,3155,35,0.98,350,644,201,...,22,53,1347.80624,1183,645.65512,151,15.25392,2,0,0
0,2018-04-18,Wednesday,2258,1354,3562,14,1.23,140,558,164,...,66,53,1437.63648,1280,640.68480,134,176.96448,24,0,0
0,2018-04-19,Thursday,2390,1354,3331,28,1.18,280,627,221,...,34,53,1382.19992,1218,911.81184,208,90.65456,12,0,0
0,2018-04-20,Friday,2127,1353,5892,30,2.15,300,519,281,...,5,53,1646.02109,1265,414.39608,107,0.00000,0,0,0
0,2018-04-21,Saturday,2307,1353,5479,18,2.05,180,626,238,...,63,55,1319.85400,1015,675.95400,146,56.40000,8,0,0
0,2018-04-22,Sunday,1968,1353,6176,16,2.51,160,1205,195,...,5,55,1569.74794,1224,224.33026,50,21.33346,3,0,0
0,2018-04-23,Monday,1985,1353,9759,25,4.21,250,571,139,...,30,55,1330.38088,1219,525.81810,112,29.88564,4,0,0
0,2018-04-24,Tuesday,2216,1353,2904,18,0.97,180,365,195,...,39,54,1414.06056,1268,684.85336,146,103.93082,14,0,0


In [57]:
# save activity summary stats into csv
acsummaryallordered_df.to_csv('./downloads/acsummaryall.csv',
                              header=True,
                              index=False)

#### 4.1.2 Periodic run to append new daily log to exported csv
- Data for each day in time period are saved into a new row.
- Use this for new daily logs.

In [13]:
# collect activity summary stats and append to existing csv

# insert date for last collected record
last_record = datetime.date(2018, 5, 2)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - last_record
delta_days = delta.days


# range does not include date of last record and iterates from oldest date to newest
for i in range(delta_days-1, 0, -1):
    date_dt = datetime.datetime.now() - datetime.timedelta(days = i)
    date = str(date_dt.strftime('%Y-%m-%d'))
    
    # activity records may not be present for every day, so if there's an IndexError, we'll skip that date
    try:
        act_stats = auth2_client.activities(date=date)['summary']
        HR_stats = auth2_client.intraday_time_series('activities/heart', base_date=date)
    except IndexError:
        print('No activity summary records for {}'.format(date))
        pass
    
    # take stats into list
    # if record is simplified (missing data), we'll skip it with except keyerror below
    try:
        acsummary_list = []
        
        acsummary_list.append(date)
        acsummary_list.append(date_dt.strftime('%A'))
        
        acsummary_list.append(act_stats['calories']['total'])
        acsummary_list.append(act_stats['calories']['bmr'])
        acsummary_list.append(act_stats['steps'])
        acsummary_list.append(act_stats['floors'])
        acsummary_list.append(act_stats['distance'])
        acsummary_list.append(act_stats['elevation'])
        acsummary_list.append(act_stats['activityLevels'][0]['minutes'])
        acsummary_list.append(act_stats['activityLevels'][1]['minutes'])
        acsummary_list.append(act_stats['activityLevels'][2]['minutes'])
        acsummary_list.append(act_stats['activityLevels'][3]['minutes'])
        acsummary_list.append(HR_stats['activities-heart'][0]['value']['restingHeartRate'])
        acsummary_list.append(act_stats['heartRateZones'][0]['caloriesOut'])
        acsummary_list.append(act_stats['heartRateZones'][0]['minutes'])
        acsummary_list.append(act_stats['heartRateZones'][1]['caloriesOut'])
        acsummary_list.append(act_stats['heartRateZones'][1]['minutes'])
        acsummary_list.append(act_stats['heartRateZones'][2]['caloriesOut'])
        acsummary_list.append(act_stats['heartRateZones'][2]['minutes'])
        acsummary_list.append(act_stats['heartRateZones'][3]['caloriesOut'])
        acsummary_list.append(act_stats['heartRateZones'][3]['minutes'])
        
        
        # convert list to numpy array and transpose
        acsummary_np = np.asarray(acsummary_list)
        new_csv_row = acsummary_np.T
        
        # add new row to existing csv
        with open(r'./downloads/acsummaryall.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_csv_row)
            print('Row added for daily activity from {}'.format(date))

            
    except KeyError:
        print('Incomplete activity records for {}'.format(date))
        pass
    

Row added for daily activity from 2018-05-03
Row added for daily activity from 2018-05-04


### 4.2 Detailed Activity Stats Data Collection
#### 4.2.1 Initial run to create dataframe and export to csv
- Data for each activity logged for each day in time period are saved into a new row.
- Need to run only once to get csv file. Then use subsequent method for appending rows for new activity logs.

In [62]:
# collect detailed activity stats for a period of time
beginning_time = datetime.date(2018, 4, 14)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - beginning_time
delta_days = delta.days


for i in range(1, delta_days):
    date = str((datetime.datetime.now() - datetime.timedelta(days = i)).strftime('%Y-%m-%d'))
    
    # activity records may not be present for every day, so if there's an IndexError, we'll skip that date
    try:
        act_details = auth2_client.activities(date=date)['activities']
        
    except IndexError:
        print('No activity details for {}. You need to exercise MORE?'.format(date))
        pass
    
    for ar in range(0, len(act_details)):
        # uncomment to catch errors/follow progress 
        ##print('This is activity index {} from {}'.format(ar, date))
        
        # take stats into dataframe
        # if record is simplified (missing data/different activity), we'll skip it with except keyerror below
        try:
            acdetails_df = pd.DataFrame({
                'Date':date,
                'Time':act_details[ar]['startTime'][11:19],
            
                'Name':act_details[ar]['activityName'],
                'Calories':act_details[ar]['calories'],
            
                #'Elevation_foot':act_details[ar]['elevationGain'], # not present in aerobic workout
                #'Distance_mile':act_details[ar]['distance'], # not present in yoga
                #'Steps':act_details[ar]['steps'], # not present in outdoor bike (2018-04-19)
                #'Speed':act_details[ar]['speed'], # not present in yoga
            
                'Duration':act_details[ar]['duration'],
                'Active_Duration':act_details[ar]['activeDuration'],
            
                'Sedentary_min':act_details[ar]['activityLevel'][0]['minutes'],
                'Light_min':act_details[ar]['activityLevel'][1]['minutes'],
                'Moderate_min':act_details[ar]['activityLevel'][2]['minutes'],
                'Very_min':act_details[ar]['activityLevel'][3]['minutes'],
            
                'HR_mean':act_details[ar]['averageHeartRate'],
                'HR_outrange_min':act_details[ar]['heartRateZones'][0]['minutes'],
                'HR_fatburn_min':act_details[ar]['heartRateZones'][1]['minutes'],
                'HR_cardio_min':act_details[ar]['heartRateZones'][2]['minutes'],
                'HR_peak_min':act_details[ar]['heartRateZones'][3]['minutes']
            } ,index=[0])
        
            if ar == 0:
                acdetailsday_df = acdetails_df
            else:
                acdetailsday_df = pd.concat([acdetails_df, acdetailsday_df])
            
        except KeyError:
            print('Incomplete activity records for {}'.format(date))
            pass 
            
    
    # combine all stats in single dataframe
    if i == 1:
        acall_df = acdetailsday_df
    else:
        acall_df = pd.concat([acdetailsday_df, acall_df])
        

In [63]:
# add day of the week to the dataframe
acall_df['Date'] = pd.to_datetime(acall_df['Date'])
acall_df['DOW'] = acall_df['Date'].dt.weekday_name

In [64]:
cols_ordered_actdetails = ['Date',
                           'DOW',
                           'Time',
                           'Name',
                           'Calories',
                           'Duration',
                           'Active_Duration',
                           'Sedentary_min',
                           'Light_min',
                           'Moderate_min',
                           'Very_min',
                           'HR_mean', 
                           'HR_outrange_min', # HR out of range: min 30, max 96
                           'HR_fatburn_min', # HR fat burn: min 96, max 135
                           'HR_cardio_min', # HR cardio: min 135, max 164
                           'HR_peak_min', # HR peak: min 164, max 220
                          ]
acallordered_df = acall_df[cols_ordered_actdetails]

In [65]:
acallordered_df

,Date,DOW,Time,Name,Calories,Duration,Active_Duration,Sedentary_min,Light_min,Moderate_min,Very_min,HR_mean,HR_outrange_min,HR_fatburn_min,HR_cardio_min,HR_peak_min
0,2018-04-15,Sunday,17:40:54,Sport,70,1382000,1382000,2,21,0,0,88,17,6,0,0
0,2018-04-15,Sunday,07:24:06,Yoga,524,9834000,7929000,17,60,35,20,108,23,103,6,0
0,2018-04-16,Monday,18:45:38,Yoga,228,3702000,3685000,12,31,10,8,104,21,39,2,0
0,2018-04-17,Tuesday,17:22:14,Yoga,417,8838000,7997000,42,71,13,7,99,54,79,0,0
0,2018-04-17,Tuesday,17:09:05,Bike,23,283000,279000,0,4,0,0,123,0,4,0,0
0,2018-04-17,Tuesday,08:45:41,Outdoor Bike,82,973000,973000,0,5,5,6,106,6,8,2,0
0,2018-04-17,Tuesday,07:28:02,Outdoor Bike,60,973000,973000,4,3,7,2,98,6,10,0,0
0,2018-04-18,Wednesday,18:52:17,Yoga,668,8502000,8303000,22,25,28,63,116,26,88,24,0
0,2018-04-19,Thursday,17:28:12,Yoga,307,7953000,6305000,37,67,0,1,98,47,58,0,0
0,2018-04-19,Thursday,07:38:34,Workout,388,5090000,5087000,1,24,42,17,112,5,77,3,0


In [66]:
# save activity details into csv with today's date
acallordered_df.to_csv('./downloads/acdetailsall.csv',
                       header=True,
                       index=False)

#### 4.2.2 Periodic run to append new activity log to exported csv
- Data for activity for each day in time period are saved into a new row.
- Use this for new activity logs.

In [14]:
# collect detailed activity stats and append to existing csv

# insert date for last collected record
last_record = datetime.date(2018, 5, 2)

# today's date
today_date = datetime.date.today()

# number of days between beginning of time and today
delta = today_date - last_record
delta_days = delta.days

# range does not include date of last record and iterates from oldest date to newest
for i in range(delta_days-1, 0, -1):
    date_dt = datetime.datetime.now() - datetime.timedelta(days = i)
    date = str(date_dt.strftime('%Y-%m-%d'))
    
    # activity records may not be present for every day, so if there's an IndexError, we'll skip that date
    try:
        act_details = auth2_client.activities(date=date)['activities']
        
    except IndexError:
        print('No activity details for {}. You need to exercise MORE?'.format(date))
        pass
    
    for ar in range(0, len(act_details)):
        # uncomment to catch errors/follow progress 
        ##print('This is activity index {} from {}'.format(ar, date))
        
        # take stats into list
        # if record is simplified (missing data/different activity), we'll skip it with except keyerror below
        try:
            acdetails_list = []
            
            acdetails_list.append(date)
            acdetails_list.append(date_dt.strftime('%A'))
            acdetails_list.append(act_details[ar]['startTime'][11:19])
            acdetails_list.append(act_details[ar]['activityName'])
            acdetails_list.append(act_details[ar]['calories'])
            acdetails_list.append(act_details[ar]['duration'])
            acdetails_list.append(act_details[ar]['activeDuration'])
            acdetails_list.append(act_details[ar]['activityLevel'][0]['minutes'])
            acdetails_list.append(act_details[ar]['activityLevel'][1]['minutes'])
            acdetails_list.append(act_details[ar]['activityLevel'][2]['minutes'])
            acdetails_list.append(act_details[ar]['activityLevel'][3]['minutes'])
            acdetails_list.append(act_details[ar]['averageHeartRate'])
            acdetails_list.append(act_details[ar]['heartRateZones'][0]['minutes'])
            acdetails_list.append(act_details[ar]['heartRateZones'][1]['minutes'])
            acdetails_list.append(act_details[ar]['heartRateZones'][2]['minutes'])
            acdetails_list.append(act_details[ar]['heartRateZones'][3]['minutes'])
            
            # convert list to numpy array and transpose
            acdetails_np = np.asarray(acdetails_list)
            new_csv_row = acdetails_np.T
        
            # add new row to existing csv
            with open(r'./downloads/acdetailsall.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(new_csv_row)
                print('Row added for daily activity for {}, index {}'.format(date, ar))
            
        except KeyError:
            print('Incomplete activity records for {}, index {}'.format(date, ar))
            pass 
            

Row added for daily activity for 2018-05-03, index 0
Row added for daily activity for 2018-05-03, index 1


> Happy coding! And don't forget to go for a walk from time to time... there's a whole world out there <3